In [1]:
from utils.utils import generator
import tensorflow as tf
import glob

In [2]:
# Get the list of file names
files = glob.glob('video/train*.tfrecord')
val = glob.glob('video/val*.tfrecord')

In [3]:
file_lens = [len([1 for example in tf.python_io.tf_record_iterator(files[0])]), 
             len([1 for example in tf.python_io.tf_record_iterator(files[1])])]
print('Steps_per_epoch should be length of data / batch size.')
print('In this case, we can choose : {} / {} (default batch_size) =~ {}'.format(min(file_lens), 32, round(min(file_lens)/32)))

Steps_per_epoch should be length of data / batch size.
In this case, we can choose : 1015 / 32 (default batch_size) =~ 32


In [4]:
def simple_model(input_shape, num_classes=3862):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(1024, activation='relu', input_shape=input_shape))
    model.add(tf.keras.layers.Dense(num_classes, activation='sigmoid'))
    return model

In [5]:
model = simple_model((1024, ))  # rgb feature size
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['top_k_categorical_accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              1049600   
_________________________________________________________________
dense_1 (Dense)              (None, 3862)              3958550   
Total params: 5,008,150
Trainable params: 5,008,150
Non-trainable params: 0
_________________________________________________________________


In [6]:
train_gen = generator(files, sel='rgb')
val_gen = generator(val, sel='rgb')
model.fit_generator(generator=train_gen, steps_per_epoch=30, epochs=10, 
                    validation_data=val_gen, validation_steps=8)

Epoch 1/10
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
30/30 [==============================] - 10s 331ms/step - loss: 0.2202 - top_k_categorical_accuracy: 0.1906 - val_loss: 0.0383 - val_top_k_categorical_accuracy: 0.4492
Epoch 2/10
30/30 [==============================] - 2s 79ms/step - loss: 0.0209 - top_k_categorical_accuracy: 0.4677 - val_loss: 0.0140 - val_top_k_categorical_accuracy: 0.5000
Epoch 3/10
30/30 [==============================] - 2s 80ms/step - loss: 0.0095 - top_k_categorical_accuracy: 0.5000 - val_loss: 0.0109 - val_top_k_categorical_accuracy: 0.5352
Epoch 4/10
30/30 [==============================] - 2s 80ms/step - loss: 0.0070 - top_k_categorical_accuracy: 0.5729 - val_loss: 0.0089 - val_top_k_categorical_accuracy: 0.5859
Epoch 5/10
30/30 [==============================] - 2s 80ms/step - loss: 0.0058 - top_k_categorical_accuracy: 0.5917 - val_loss: 0.0079 - val_top_k_categorical_accuracy: 0.5898
Epoch 6/10
30/

In [7]:
def cnn_model(input_shape, num_classes=3862):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Reshape((1, input_shape[0]), input_shape=input_shape))
    model.add(tf.keras.layers.Convolution1D(256, 3, padding='same', activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(num_classes, activation='sigmoid'))
    return model

In [8]:
model = cnn_model((1024, ))  # rgb feature size
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['top_k_categorical_accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 1, 1024)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 1, 256)            786688    
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 3862)              992534    
Total params: 1,779,222
Trainable params: 1,779,222
Non-trainable params: 0
_________________________________________________________________


In [9]:
train_gen = generator(files, sel='rgb')
val_gen = generator(val, sel='rgb')
model.fit_generator(generator=train_gen, steps_per_epoch=30, epochs=10, 
                    validation_data=val_gen, validation_steps=8)

Epoch 1/10
30/30 [==============================] - 3s 97ms/step - loss: 0.5003 - top_k_categorical_accuracy: 0.0031 - val_loss: 0.2913 - val_top_k_categorical_accuracy: 0.0312
Epoch 2/10
30/30 [==============================] - 2s 59ms/step - loss: 0.1707 - top_k_categorical_accuracy: 0.2417 - val_loss: 0.1062 - val_top_k_categorical_accuracy: 0.3711
Epoch 3/10
30/30 [==============================] - 2s 60ms/step - loss: 0.0575 - top_k_categorical_accuracy: 0.4521 - val_loss: 0.0565 - val_top_k_categorical_accuracy: 0.4062
Epoch 4/10
30/30 [==============================] - 2s 59ms/step - loss: 0.0248 - top_k_categorical_accuracy: 0.4906 - val_loss: 0.0376 - val_top_k_categorical_accuracy: 0.4375
Epoch 5/10
30/30 [==============================] - 2s 60ms/step - loss: 0.0158 - top_k_categorical_accuracy: 0.4781 - val_loss: 0.0265 - val_top_k_categorical_accuracy: 0.4688
Epoch 6/10
30/30 [==============================] - 2s 63ms/step - loss: 0.0118 - top_k_categorical_accuracy: 0.478

In [10]:
def lstm_model(input_shape, num_classes=3862):
    model = tf.keras.Sequential()    
    model.add(tf.keras.layers.Reshape((1, input_shape[0]), input_shape=input_shape))
    model.add(tf.keras.layers.LSTM(128))
    model.add(tf.keras.layers.Dense(num_classes, activation='sigmoid'))
    return model

In [11]:
model = lstm_model((1024,))  # rgb feature size
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['top_k_categorical_accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 1, 1024)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 128)               590336    
_________________________________________________________________
dense_3 (Dense)              (None, 3862)              498198    
Total params: 1,088,534
Trainable params: 1,088,534
Non-trainable params: 0
_________________________________________________________________


In [12]:
train_gen = generator(files, sel='rgb')
val_gen = generator(val, sel='rgb')
model.fit_generator(generator=train_gen, steps_per_epoch=30, epochs=10, 
                    validation_data=val_gen, validation_steps=8)

Epoch 1/10
30/30 [==============================] - 3s 116ms/step - loss: 0.6849 - top_k_categorical_accuracy: 0.0021 - val_loss: 0.6729 - val_top_k_categorical_accuracy: 0.0000e+00
Epoch 2/10
30/30 [==============================] - 1s 39ms/step - loss: 0.6429 - top_k_categorical_accuracy: 0.0427 - val_loss: 0.6083 - val_top_k_categorical_accuracy: 0.1211
Epoch 3/10
30/30 [==============================] - 1s 39ms/step - loss: 0.5094 - top_k_categorical_accuracy: 0.2594 - val_loss: 0.4584 - val_top_k_categorical_accuracy: 0.3008
Epoch 4/10
30/30 [==============================] - 1s 39ms/step - loss: 0.3081 - top_k_categorical_accuracy: 0.4167 - val_loss: 0.3392 - val_top_k_categorical_accuracy: 0.4453
Epoch 5/10
30/30 [==============================] - 1s 39ms/step - loss: 0.1751 - top_k_categorical_accuracy: 0.4385 - val_loss: 0.2307 - val_top_k_categorical_accuracy: 0.4727
Epoch 6/10
30/30 [==============================] - 1s 41ms/step - loss: 0.1048 - top_k_categorical_accuracy: 